In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.express as px



In [2]:
lc_weekly=pd.read_csv('weekly_sales/lc_weekly.csv')


# lc_weekly_sale by 1po_3po
first_party=['Mobile','Website']
third_party=['DoorDash', 'UberEats','Grubhub']


lc_weekly['channel']=np.where(lc_weekly['ORDERPLACEDMETHOD'].isin(first_party),'1PO',np.where(lc_weekly['ORDERPLACEDMETHOD'].isin(third_party),'3PO',lc_weekly['ORDERPLACEDMETHOD']))
lc=lc_weekly[lc_weekly['channel'].isin(['1PO','3PO'])]
lc.head(3)


,FranchiseStore,ORDERPLACEDMETHOD,WeekStartDate,WeeklyNetAmount,channel
1,01389-00001,DoorDash,2024-02-19,775.89,3PO
2,01389-00001,UberEats,2024-02-19,74.25,3PO
3,01389-00001,Website,2024-02-19,1528.66,1PO


In [16]:
ws_weekly=pd.read_csv('weekly_sales/ws_weekly.csv')
ws_weekly.drop(columns=['Unnamed: 0','HourAMPM','StoreName'],inplace=True)
#dtyps
ws_weekly['Date']=pd.to_datetime(ws_weekly['Date'])
ws_weekly['SoldPriceAmount']=ws_weekly['SoldPriceAmount'].astype(float)


In [17]:
ws_weekly['OrderMode'].unique()


array(['Call-DineIn', 'Call-ToGo', 'CallCtr-ToGo', 'DD-Delivery',
       'Dispatch', 'Online', 'Uber Eats', 'Unknown', 'Walk Dine In',
       'Walk-ToGo', 'Delivery'], dtype=object)

In [4]:

first_party_ws=[ 'Online']
third_party_ws=[ 'DD-Delivery', 'Uber Eats']
ws_weekly

ws_weekly['channel']=np.where(ws_weekly['OrderMode'].isin(first_party_ws),'1PO',np.where(ws_weekly['OrderMode'].isin(third_party_ws),'3PO',ws_weekly['OrderMode']))


ws_weekly['OrderMode'] = np.where(
    ws_weekly['OrderMode'] == 'DD-Delivery',
    'DoorDash',
    np.where(
        ws_weekly['OrderMode'] == 'Uber Eats',
        'UberEats',
        ws_weekly['OrderMode']
    )
)

ws=ws_weekly[ws_weekly['channel'].isin(['1PO','3PO'])]
print(ws.OrderMode.unique())


['DoorDash' 'Online' 'UberEats']


In [5]:
lc.rename(columns={'FranchiseStore':'StoreID',
                   'ORDERPLACEDMETHOD':'OrderMode',
                   },inplace=True)

ws.rename(columns={'SoldPriceAmount':'WeeklyNetAmount',
                   'Date':'WeekStartDate',
                   },inplace=True)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
ws['StoreID'].nunique()


75

In [15]:
75+76


151

In [6]:
ws['Brand']='WS'

ws['revenue']=ws['WeeklyNetAmount']

ws.columns


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Index(['StoreID', 'OrderMode', 'WeekStartDate', 'WeeklyNetAmount', 'channel',
       'Brand', 'revenue'],
      dtype='object')

In [7]:
lc['Brand']='LC'
lc['revenue']=lc['WeeklyNetAmount']
lc.columns


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Index(['StoreID', 'OrderMode', 'WeekStartDate', 'WeeklyNetAmount', 'channel',
       'Brand', 'revenue'],
      dtype='object')

In [13]:
lc['StoreID'].nunique()


76

In [8]:
df_combined=pd.concat([lc,ws],axis=0)
df_combined['WeekStartDate']=pd.to_datetime(df_combined['WeekStartDate'])

df_combined.head()


,StoreID,OrderMode,WeekStartDate,WeeklyNetAmount,channel,Brand,revenue
1,01389-00001,DoorDash,2024-02-19,775.89,3PO,LC,775.89
2,01389-00001,UberEats,2024-02-19,74.25,3PO,LC,74.25
3,01389-00001,Website,2024-02-19,1528.66,1PO,LC,1528.66
6,01389-00001,Mobile,2024-02-19,2005.36,1PO,LC,2005.36
8,01389-00001,Grubhub,2024-02-26,16.12,3PO,LC,16.12


In [9]:
df_combined['revenue']=df_combined['WeeklyNetAmount']


In [10]:
df_combined.columns

# compute each group’s min & max week
min_week = df_combined.groupby(['Brand','OrderMode'])['WeekStartDate'].transform('min')
max_week = df_combined.groupby(['Brand','OrderMode'])['WeekStartDate'].transform('max')

# keep only the rows that are neither the group‐min nor the group‐max
df_combined = df_combined[(df_combined['WeekStartDate'] != min_week) & (df_combined['WeekStartDate'] != max_week)]



In [12]:
df_combined.to_csv('vibe.csv',index=False)


# charts


In [13]:
brand_colors = {
    'LC': 'rgb(246, 35, 151)',  
    'WS': 'rgb(22, 198, 206)'  
}

channel_colors={
    'Grubhub':'(14,133)',
    'Mobile':'rgb(43,75,149)', 
    'UberEats':'rgb(149,43,88)'
    'DoorDash'':rgb(209,116,10)' ,
    'Website': 'rgb(78,210,184)',
    'Online':'rgb(180,93,231)'
  
    
}

mode_colours={}


In [14]:
df_combined.OrderMode.unique()


array(['Grubhub', 'Mobile', 'UberEats', 'DoorDash', 'Website', 'Online'],
      dtype=object)

In [17]:
revenue_share=df_combined.groupby(['Brand','channel'])['revenue'].sum().reset_index()
revenue_share['rev_%'] = (
    revenue_share
      .groupby('Brand')['revenue']
      .transform(lambda x: x / x.sum() * 100)
)

# 3. (Optional) Round for readability
revenue_share['rev_%'] = revenue_share['rev_%'].round(1)


# Format percentages to 2 decimals for text labels
revenue_share['revenue_text'] = revenue_share['rev_%'].apply(lambda x: f'{x:.2f}%')
brand_colors = {
    'LC': 'rgb(246, 35, 151)',  
    'WS': 'rgb(22, 198, 206)'  
}
fig = px.bar(
    revenue_share,
    x='channel',
    y='rev_%',
    color='Brand',
    barmode='group',
    text='revenue_text',
    color_discrete_map=brand_colors,
    title='Revenue Percentage by Channel and Brand'
)

fig.update_traces(textposition='outside')

fig.update_layout(
    xaxis_title='Channel',
    yaxis_title='Revenue (%)',
    yaxis=dict(range=[1, 100])
)
fig.update_layout(
    width=800,   # Set the desired width (in pixels)
    height=500,  # Set the desired height (in pixels)
)

fig.show()


In [18]:
revenue_share


,Brand,channel,revenue,rev_%,revenue_text
0,LC,1PO,33781841.67,75.0,75.00%
1,LC,3PO,11272581.86,25.0,25.00%
2,WS,1PO,44258088.05,55.8,55.80%
3,WS,3PO,35114767.93,44.2,44.20%


In [19]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Aggregate weekly revenue
weekly = (
    df_combined[df_combined['Brand'] == 'LC']
    .groupby(['Brand', 'channel', 'OrderMode', 'WeekStartDate'], as_index=False)['revenue']
    .sum()
)

# 2. Unique brands and channels
brands = weekly['Brand'].unique().tolist()
channels = weekly['channel'].unique().tolist()
n_rows, n_cols = len(brands), len(channels)

# 3. Custom colors for each channel
channel_colors = {
    'Grubhub': 'rgb(14, 133, 133)',
    'Mobile': 'rgb(43, 75, 149)',
    'UberEats': 'rgb(149, 43, 88)',
    'DoorDash': 'rgb(209, 116, 10)',
    'Website': 'rgb(78, 210, 184)',
    'Online': 'rgb(180, 93, 231)'
}

# 4. Create subplot grid
fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    shared_xaxes=False,
    shared_yaxes=False,
    subplot_titles=[f"{b} — {c}" for b in brands for c in channels],
    vertical_spacing=0.09
)

# 5. Plotting
shown_modes = set()

for i, brand in enumerate(brands, start=1):
    for j, channel in enumerate(channels, start=1):
        cell = weekly[(weekly['Brand'] == brand) & (weekly['channel'] == channel)]
        if cell.empty:
            continue

        for mode in cell['OrderMode'].unique():
            sub = cell[cell['OrderMode'] == mode]
            x = sub['WeekStartDate']
            y = sub['revenue']

            show_leg = mode not in shown_modes
            shown_modes.add(mode)

            # # Color by channel
            # color = channel_colors.get(channel, 'gray')

            # 5a. Actual data line
            fig.add_trace(
                go.Scatter(
                    x=x, y=y,
                    mode='lines+markers',
                    name=mode,
                    showlegend=show_leg,
                    line=dict(color=color)
                ),
                row=i, col=j
            )

            # 5b. Trend line
            x_ord = x.map(pd.Timestamp.toordinal).values
            slope, intercept = np.polyfit(x_ord, y, 1)
            y_hat = slope * x_ord + intercept
            ss_res = np.sum((y - y_hat) ** 2)
            ss_tot = np.sum((y - np.mean(y)) ** 2)
            r2 = 1 - ss_res / ss_tot

            fig.add_trace(
                go.Scatter(
                    x=x, y=y_hat,
                    mode='lines',
                    line=dict(dash='dash', color='green'),
                    showlegend=False
                ),
                row=i, col=j
            )

            # 5c. Annotate with slope & R²
            sp_idx = (i - 1) * n_cols + j
            xref = f"x{sp_idx}" if sp_idx > 1 else "x"
            yref = f"y{sp_idx}" if sp_idx > 1 else "y"

            fig.add_annotation(
                xref=xref, yref=yref,
                x=min(x), y=max(y_hat),
                text=f"slope={slope:.2f}<br>R²={r2:.2f}",
                showarrow=False,
                font=dict(size=10, color="black"),
                bgcolor="rgba(255,255,255,0.7)",
                xanchor="left", yanchor="top"
            )

# 6. Final layout tweaks
fig.update_yaxes(matches=None)
fig.update_layout(
    height=450 * n_rows,
    width=600 * n_cols,
    title="Weekly Revenue with Trend Lines & R²",
    legend_title_text="OrderMode"
)

fig.write_image("2.png")
fig.show()


NameError: name 'color' is not defined

In [20]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


weekly = (
    df_combined[df_combined['Brand']=='LC']
      .groupby(['Brand','channel','OrderMode','WeekStartDate'], as_index=False)
      ['revenue'].sum()
)

brands   = weekly['Brand'].unique().tolist()
channels = weekly['channel'].unique().tolist()
n_rows, n_cols = len(brands), len(channels)

# 2) Create subplot grid
fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    shared_xaxes=False,
    shared_yaxes=False,
    subplot_titles=[f"{b} — {c}" for b in brands for c in channels],
     vertical_spacing=0.09
      
)

# 3) Plot data, trendlines, legend & annotations
shown_modes = set()

for i, brand in enumerate(brands, start=1):
    for j, channel in enumerate(channels, start=1):
        cell = weekly[(weekly['Brand']==brand) & (weekly['channel']==channel)]
        if cell.empty:
            continue

        for mode in cell['OrderMode'].unique():
            sub = cell[cell['OrderMode']==mode]
            x = sub['WeekStartDate']
            y = sub['revenue']

            # only show each OrderMode once in the legend
            show_leg = mode not in shown_modes
            shown_modes.add(mode)

            # 3a) Data line
            fig.add_trace(
                go.Scatter(
                    
                    x=x, y=y,
                    mode='lines+markers',
                    name=mode,
                    showlegend=show_leg

                ),
                row=i, col=j
            )

            # 3b) Compute trend & R²
            x_ord = x.map(pd.Timestamp.toordinal).values
            slope, intercept = np.polyfit(x_ord, y, 1)
            y_hat = slope * x_ord + intercept
            ss_res = np.sum((y - y_hat)**2)
            ss_tot = np.sum((y - np.mean(y))**2)
            r2 = 1 - ss_res/ss_tot

            # 3c) Trend‐line (green dashed)
            fig.add_trace(
                go.Scatter(
                    x=x, y=y_hat,
                    mode='lines',
                    line=dict(dash='dash', color='green'),
                    showlegend=False
                ),
                row=i, col=j
            )

            # 3d) Annotate with slope & R²
            # Calculate subplot index for xref/yref:
            sp_idx = (i-1)*n_cols + j
            xref = f"x{sp_idx}" if sp_idx>1 else "x"
            yref = f"y{sp_idx}" if sp_idx>1 else "y"

            fig.add_annotation(
                xref=xref, yref=yref,
                x=min(x), y=max(y_hat),
                text=f"slope={slope:.2f}<br>R²={r2:.2f}",
                showarrow=False,
                font=dict(size=10, color="black"),
                bgcolor="rgba(255,255,255,0.7)",
                xanchor="left", yanchor="top"
            )

# 4) Final layout tweaks
fig.update_yaxes(matches=None)
fig.update_layout(
    height=450 * n_rows,
    width=600 * n_cols,
    title="Weekly Revenue with Trend Lines & R²",
    legend_title_text="OrderMode"
)
fig.write_image("2.png")
fig.show()


In [21]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


weekly = (
    df_combined[df_combined['Brand']=='WS']
      .groupby(['Brand','channel','OrderMode','WeekStartDate'], as_index=False)
      ['revenue'].sum()
)

brands   = weekly['Brand'].unique().tolist()
channels = weekly['channel'].unique().tolist()
n_rows, n_cols = len(brands), len(channels)

# 2) Create subplot grid
fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    shared_xaxes=False,
    shared_yaxes=False,
    subplot_titles=[f"{b} — {c}" for b in brands for c in channels],
     vertical_spacing=0.09
)

# 3) Plot data, trendlines, legend & annotations
shown_modes = set()

for i, brand in enumerate(brands, start=1):
    for j, channel in enumerate(channels, start=1):
        cell = weekly[(weekly['Brand']==brand) & (weekly['channel']==channel)]
        if cell.empty:
            continue

        for mode in cell['OrderMode'].unique():
            sub = cell[cell['OrderMode']==mode]
            x = sub['WeekStartDate']
            y = sub['revenue']

            # only show each OrderMode once in the legend
            show_leg = mode not in shown_modes
            shown_modes.add(mode)

            # 3a) Data line
            fig.add_trace(
                go.Scatter(
                    x=x, y=y,
                    mode='lines+markers',
                    name=mode,
                    showlegend=show_leg
                ),
                row=i, col=j
            )

            # 3b) Compute trend & R²
            x_ord = x.map(pd.Timestamp.toordinal).values
            slope, intercept = np.polyfit(x_ord, y, 1)
            y_hat = slope * x_ord + intercept
            ss_res = np.sum((y - y_hat)**2)
            ss_tot = np.sum((y - np.mean(y))**2)
            r2 = 1 - ss_res/ss_tot

            # 3c) Trend‐line (green dashed)
            fig.add_trace(
                go.Scatter(
                    x=x, y=y_hat,
                    mode='lines',
                    line=dict(dash='dash', color='green'),
                    showlegend=False
                ),
                row=i, col=j
            )

            # 3d) Annotate with slope & R²
            # Calculate subplot index for xref/yref:
            sp_idx = (i-1)*n_cols + j
            xref = f"x{sp_idx}" if sp_idx>1 else "x"
            yref = f"y{sp_idx}" if sp_idx>1 else "y"

            fig.add_annotation(
                xref=xref, yref=yref,
                x=min(x), y=max(y_hat),
                text=f"slope={slope:.2f}<br>R²={r2:.2f}",
                showarrow=False,
                font=dict(size=10, color="black"),
                bgcolor="rgba(255,255,255,0.7)",
                xanchor="left", yanchor="top"
            )

# 4) Final layout tweaks
fig.update_yaxes(matches=None)
fig.update_layout(
    height=450 * n_rows,
    width=600 * n_cols,
    title="Weekly Revenue with Trend Lines & R²",
    legend_title_text="OrderMode"
)
fig.write_image("2.png")
fig.show()


In [22]:
import pandas as pd
import plotly.graph_objects as go

# 1. Aggregate revenue by Brand → Channel → OrderMode
agg = lc.groupby(['Brand', 'channel', 'OrderMode'], as_index=False)['revenue'].sum()

# 2. Prepare label lists
brands = agg['Brand'].unique().tolist()
channels = agg['channel'].unique().tolist()
modes = agg['OrderMode'].unique().tolist()

labels = brands + channels + modes  # Ensure order is Brand → Channel → Mode

# 3. Map labels to indices
label_indices = {label: i for i, label in enumerate(labels)}

# 4. Create Sankey links
source = []
target = []
value = []
custom_labels = []

# Calculate total revenue for percentage calculation
total_revenue = agg['revenue'].sum()

# Brand → Channel links
brand_channel_agg = agg.groupby(['Brand', 'channel'], as_index=False)['revenue'].sum()
for _, row in brand_channel_agg.iterrows():
    percent = (row['revenue'] / total_revenue) * 100
    source.append(label_indices[row['Brand']])
    target.append(label_indices[row['channel']])
    value.append(row['revenue'])
    custom_labels.append(
        f"{row['Brand']} → {row['channel']}<br>"
        f"Revenue: ${row['revenue']:,.2f}<br>"
        f"Share: {percent:.2f}%"
    )

# Channel → OrderMode links
for _, row in agg.iterrows():
    percent = (row['revenue'] / total_revenue) * 100
    source.append(label_indices[row['channel']])
    target.append(label_indices[row['OrderMode']])
    value.append(row['revenue'])
    custom_labels.append(
        f"{row['channel']} → {row['OrderMode']}<br>"
        f"Revenue: ${row['revenue']:,.2f}<br>"
        f"Share: {percent:.2f}%"
    )

# 5. Plot Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        customdata=custom_labels,
        hovertemplate='%{customdata}<extra></extra>'
    )
))

fig.update_layout(
    title_text="Revenue% Flow for Little Caesars: Brand → Channel → Order Mode",
    font_size=12,
  
    width=1200,     # adjust width
    height=400      # adjust height
)


# Optional: Save image
fig.write_image("3.png")

# Show the plot
fig.show()


In [23]:
47+27+20+4


98

In [24]:
import pandas as pd
import plotly.graph_objects as go

# 1. Aggregate revenue by Brand → Channel → OrderMode
agg = ws.groupby(['Brand', 'channel', 'OrderMode'], as_index=False)['revenue'].sum()

# 2. Prepare label lists
brands = agg['Brand'].unique().tolist()
channels = agg['channel'].unique().tolist()
modes = agg['OrderMode'].unique().tolist()

labels = brands + channels + modes  # Ensure order is Brand → Channel → Mode

# 3. Map labels to indices
label_indices = {label: i for i, label in enumerate(labels)}

# 4. Create Sankey links
source = []
target = []
value = []
custom_labels = []

# Calculate total revenue for percentage calculation
total_revenue = agg['revenue'].sum()

# Brand → Channel links
brand_channel_agg = agg.groupby(['Brand', 'channel'], as_index=False)['revenue'].sum()
for _, row in brand_channel_agg.iterrows():
    percent = (row['revenue'] / total_revenue) * 100
    source.append(label_indices[row['Brand']])
    target.append(label_indices[row['channel']])
    value.append(row['revenue'])
    custom_labels.append(
        f"{row['Brand']} → {row['channel']}<br>"
        f"Revenue: ${row['revenue']:,.2f}<br>"
        f"Share: {percent:.2f}%"
    )

# Channel → OrderMode links
for _, row in agg.iterrows():
    percent = (row['revenue'] / total_revenue) * 100
    source.append(label_indices[row['channel']])
    target.append(label_indices[row['OrderMode']])
    value.append(row['revenue'])
    custom_labels.append(
        f"{row['channel']} → {row['OrderMode']}<br>"
        f"Revenue: ${row['revenue']:,.2f}<br>"
        f"Share: {percent:.2f}%"
    )

# 5. Plot Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        customdata=custom_labels,
        hovertemplate='%{customdata}<extra></extra>'
    )
))

fig.update_layout(
    title_text="Revenue% Flow for Wingstop: Brand → Channel → Order Mode",
    font_size=12,
  
    width=1200,     # adjust width
    height=400      # adjust height
)


# Optional: Save image
fig.write_image("3.png")

# Show the plot
fig.show()


In [27]:
ws_location=pd.read_csv('2025 Vibe Wingstop Locations.csv')
ws_location.columns


Index(['Wing Group', 'Store ID', 'Address', 'City', 'State', 'Postal Code',
       'Unnamed: 6'],
      dtype='object')

In [28]:
ws_location['Store ID'].unique()


array(['2133', '1819', '2174', '2836', '2884', '2888', '4019', '*0024',
       '*0899', '1614', '1859', '2630', '**2765**', '*0081', '*0444',
       '1190', '2212', '2252', '*0436', '*0084', '1151', '*0031', '1141',
       '2764', '3066', '*0064', '*0448', '*0524', '1260', '1458', '1724',
       '1142', '2114', '2440', '2519', '2579', '2691', '1010', '2153',
       '2220', '2291', '2851', '2874', '1198', '1382', '2207', '\n2760',
       '*0212', '2572', '1607', '2879', '\n**4015**', '**3009**', '*0617',
       '1367', '2442', '2655', '1038', '1757', '2687', 'TBD', '2189',
       '1858', '*0892', '\n1144', '*0117', '1579', '1733', '2854',
       '*0542', '2047', '2580', '2603', '2629', '2961', '*0778', '1766',
       '*0838', '1998', '2177', '2303'], dtype=object)

In [29]:
# Step-by-step cleaning:
ws_location['cleaned_ids'] = (
    ws_location['Store ID']
    .str.replace(r'\D', '', regex=True)      # Remove all non-digit characters
    .str.lstrip('0')                         # Strip leading zeros
    .replace('', '0')                        # Handle empty strings if any
    .astype(int)                           # Convert to float
)

ws_location['cleaned_ids'].unique()


array([2133, 1819, 2174, 2836, 2884, 2888, 4019,   24,  899, 1614, 1859,
       2630, 2765,   81,  444, 1190, 2212, 2252,  436,   84, 1151,   31,
       1141, 2764, 3066,   64,  448,  524, 1260, 1458, 1724, 1142, 2114,
       2440, 2519, 2579, 2691, 1010, 2153, 2220, 2291, 2851, 2874, 1198,
       1382, 2207, 2760,  212, 2572, 1607, 2879, 4015, 3009,  617, 1367,
       2442, 2655, 1038, 1757, 2687,    0, 2189, 1858,  892, 1144,  117,
       1579, 1733, 2854,  542, 2047, 2580, 2603, 2629, 2961,  778, 1766,
        838, 1998, 2177, 2303])

In [30]:
ws['StoreID'].unique()


array([  24,   31,   64,   81,   84,  117,  212,  436,  444,  448,  524,
        542,  617,  778,  838,  892,  899, 1010, 1038, 1141, 1142, 1144,
       1151, 1190, 1198, 1260, 1367, 1382, 1458, 1579, 1607, 1614, 1724,
       1733, 1757, 1766, 1819, 1858, 1859, 1998, 2047, 2114, 2133, 2153,
       2174, 2177, 2189, 2207, 2212, 2220, 2252, 2291, 2303, 2440, 2442,
       2519, 2572, 2579, 2580, 2603, 2629, 2630, 2655, 2687, 2691, 2760,
       2764, 2836, 2851, 2854, 2874, 2879, 2884, 2888, 2961])

In [31]:
ws_location.columns


Index(['Wing Group', 'Store ID', 'Address', 'City', 'State', 'Postal Code',
       'Unnamed: 6', 'cleaned_ids'],
      dtype='object')

In [32]:

df1=ws_location[['State', 'cleaned_ids', 'City']]
df1['StoreID']=df1['cleaned_ids']
df1.rename(columns={'State':'ST'},inplace=True)


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_77849/191059037.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_77849/191059037.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
ws_extended=ws.merge(df1,on='StoreID',how='left')
ws_extended.columns


Index(['StoreID', 'OrderMode', 'WeekStartDate', 'WeeklyNetAmount', 'channel',
       'Brand', 'revenue', 'ST', 'cleaned_ids', 'City'],
      dtype='object')

In [37]:
lc_location=pd.read_excel('VIBE_LC locations.xlsx')
# Ensure the column is float (if it isn't already)
lc_location['Franchise Store'] = lc_location['Franchise Store'].astype(float)

# Split the number into integer and decimal parts
lc_location['store_id_formatted'] =lc_location['Franchise Store'] .apply(
    lambda x: f"{int(x):05d}-{int(round((x - int(x)) * 10000)):05d}"
)
df=lc_location[['ST','store_id_formatted','City','Zone','Latitude', 'Longitude']]
df['StoreID']=df['store_id_formatted']
lc_extended=lc.merge(df,on='StoreID',how='left')


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_77849/3621048963.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
x=pd.concat([ws_extended,lc_extended],axis=0)


In [39]:
x['Brand'].unique()


array(['WS', 'LC'], dtype=object)

In [40]:
import pandas as pd
import plotly.express as px

# Assuming aux is your DataFrame already preloaded and cleaned

# Grouping the data
aux = x[x['Brand']=='LC'].groupby(['Brand', 'ST', 'channel', 'OrderMode'], as_index=False)['revenue'].sum()

# Calculating revenue share within each channel
total_rev_per_channel = aux.groupby('channel')['revenue'].transform('sum')
aux['revenue_share'] = aux['revenue'] / total_rev_per_channel * 100
aux['revenue_share_label'] = aux['revenue_share'].map(lambda x: f"{x:.1f}%")

# Optional: Custom colors
OrderMode_c = {
    'Grubhub': '#EF553B',
    'Mobile': '#636EFA',
    'UberEats': '#FF6692',
    'DoorDash': '#FFA15A',
    'Website': 'rgb(68, 170, 153)',
    'Online': 'rgb(144, 103, 167)'
}

# Creating the plot
fig = px.bar(
    aux,
    x="ST",
    y="revenue_share",
    color="OrderMode",
    text='revenue_share_label',
    title="State Revenue Share by Order Mode (Faceted by Channel and Brand)",
    labels={"ST": "State", "revenue_share": "Revenue Share (%)"},
   
    facet_col="Brand",
     facet_row="channel",
    color_discrete_map=OrderMode_c,
    height=800,
    width=1000
)

fig.update_layout(
    yaxis_title="Revenue Share (%)",
    showlegend=True
)

fig.update_yaxes(matches=None)

fig.show()


In [41]:
aux[aux['ST']=='SD']


,Brand,ST,channel,OrderMode,revenue,revenue_share,revenue_share_label


In [42]:
import pandas as pd
import plotly.express as px

# Assuming aux is your DataFrame already preloaded and cleaned

# Grouping the data
aux = x[x['Brand']=='WS'].groupby(['Brand', 'ST', 'channel', 'OrderMode'], as_index=False)['revenue'].sum()

# Calculating revenue share within each channel
total_rev_per_channel = aux.groupby('channel')['revenue'].transform('sum')
aux['revenue_share'] = aux['revenue'] / total_rev_per_channel * 100
aux['revenue_share_label'] = aux['revenue_share'].map(lambda x: f"{x:.1f}%")

# Optional: Custom colors
OrderMode_c = {
    'Grubhub': '#EF553B',
    'Mobile': '#636EFA',
    'UberEats': '#FF6692',
    'DoorDash': '#FFA15A',
    'Website': 'rgb(68, 170, 153)',
    'Online': 'rgb(144, 103, 167)'
}

# Creating the plot
fig = px.bar(
    aux,
    x="ST",
    y="revenue_share",
    color="OrderMode",
    text='revenue_share_label',
    title="State Revenue Share by Order Mode (Faceted by Channel and Brand)",
    labels={"ST": "State", "revenue_share": "Revenue Share (%)"},
   
    facet_col="Brand",
     facet_row="channel",
    color_discrete_map=OrderMode_c,
    height=800,
    width=1000
)

fig.update_layout(
    yaxis_title="Revenue Share (%)",
    showlegend=True
)

fig.update_yaxes(matches=None)

fig.show()


In [43]:
aux[(aux['Brand']=='LC' )]['revenue_share'].sum()


np.float64(0.0)

In [44]:
x.columns


Index(['StoreID', 'OrderMode', 'WeekStartDate', 'WeeklyNetAmount', 'channel',
       'Brand', 'revenue', 'ST', 'cleaned_ids', 'City', 'store_id_formatted',
       'Zone', 'Latitude', 'Longitude'],
      dtype='object')

In [45]:
import pandas as pd
import plotly.express as px

# 1. Filter for Little Caesars and calculate revenue per state/channel/ordermode
aux = x[x['Brand'] == 'LC'].groupby(['Brand', 'ST', 'channel', 'OrderMode','StoreID'], as_index=False)['revenue'].sum()

# 2. Get store count per state (assuming 'store_id' column exists in your dataset)
store_counts = x[x['Brand'] == 'LC'].groupby('ST')['StoreID'].nunique().reset_index()
store_counts.rename(columns={'StoreID': 'store_count'}, inplace=True)
store_counts


,ST,store_count
0,AR,1
1,KS,14
2,LA,2
3,MD,6
4,NC,7
5,NM,6
6,OK,17
7,TX,20
8,VA,3


In [46]:
# 3. Merge store count into the main data
aux = aux.merge(store_counts, on='ST', how='left')
aux=aux[['Brand','ST','channel','OrderMode','revenue']]
aux.shape
# 4. Calculat


(344, 5)

In [47]:
aux=aux.groupby(['Brand','ST',	'channel',	'OrderMode']).sum().reset_index()


In [48]:
total_rev_per_state= aux.groupby('ST')['revenue'].transform('sum')
total_rev_per_state
aux['total_rev_per_state']=total_rev_per_state
aux[aux['ST']=='VA']


,Brand,ST,channel,OrderMode,revenue,total_rev_per_state
39,LC,VA,1PO,Mobile,758383.79,2174450.45
40,LC,VA,1PO,Website,539627.87,2174450.45
41,LC,VA,3PO,DoorDash,504888.66,2174450.45
42,LC,VA,3PO,Grubhub,54744.81,2174450.45
43,LC,VA,3PO,UberEats,316805.32,2174450.45


In [49]:


aux['revenue Share(%)']=(aux['revenue']/aux['total_rev_per_state'])*100
aux


aux['revenue_share_label'] = aux['revenue Share(%)'].map(lambda x: f"{x:.1f}%")

aux


,Brand,ST,channel,OrderMode,revenue,total_rev_per_state,revenue Share(%),revenue_share_label
0,LC,AR,1PO,Mobile,217804.88,405627.39,53.695802,53.7%
1,LC,AR,1PO,Website,111642.11,405627.39,27.523316,27.5%
2,LC,AR,3PO,DoorDash,73393.18,405627.39,18.093744,18.1%
3,LC,AR,3PO,UberEats,2787.22,405627.39,0.687138,0.7%
4,LC,KS,1PO,Mobile,4048533.92,8002381.52,50.591613,50.6%
5,LC,KS,1PO,Website,2430463.82,8002381.52,30.371756,30.4%
6,LC,KS,3PO,DoorDash,1277348.38,8002381.52,15.962103,16.0%
7,LC,KS,3PO,Grubhub,40071.55,8002381.52,0.500745,0.5%
8,LC,KS,3PO,UberEats,205963.85,8002381.52,2.573782,2.6%
9,LC,LA,1PO,Mobile,347011.96,879762.27,39.443833,39.4%


In [50]:
aux[aux['ST']=='KS']['revenue Share(%)'].sum()


np.float64(100.0)

In [51]:

# 4. Plot with custom colors
fig = px.bar(aux, 
             x="ST", 
             y="revenue Share(%)", 
             color="OrderMode",
             text='revenue_share_label',
             title="Little Caesars Revenue Share(%) by State",
             labels={"ST": "State"},
             facet_row="channel",
            #  color_discrete_map=OrderMode_c,  # <-- Use custom colors here
             height=700,
             width=900)

fig.update_layout(
    yaxis_title="Revenue Share(%)",
    showlegend=True
)

# Adjust font size for various elements
fig.update_layout(
    yaxis_title="Share per Store (%)",
    showlegend=True,
    title_font=dict(size=16),  # Title font size
    font=dict(size=10),  # Axis labels and general text size
    xaxis_title_font=dict(size=12),  # X-axis label font size
    yaxis_title_font=dict(size=12),  # Y-axis label font size
    legend_title_font=dict(size=12),  # Legend title font size
    legend_font=dict(size=10),  # Legend text size
)
fig.update_yaxes(matches=None)
fig.update_yaxes(showticklabels=True)  # Ensure y-axis labels show on one axis
fig.update_yaxes(title_text="", row=1, col=1)  # Remove y-axis title for each facet


fig.show()




In [391]:

# 3. Merge store count into the main data
aux = aux.merge(store_counts, on='ST', how='left')
aux.head()
# 4. Calculat


,Brand,ST,channel,OrderMode,StoreID,revenue,store_count
0,LC,AR,1PO,Mobile,01494-00005,217804.88,1
1,LC,AR,1PO,Website,01494-00005,111642.11,1
2,LC,AR,3PO,DoorDash,01494-00005,73393.18,1
3,LC,AR,3PO,UberEats,01494-00005,2787.22,1
4,LC,KS,1PO,Mobile,03222-00009,141294.58,14


In [385]:
mean_rev=aux.groupby(['ST','channel','OrderMode']).mean().reset_index()
mean_rev.head()


,ST,channel,OrderMode,revenue,store_count
0,AR,1PO,Mobile,217804.880000,1.0
1,AR,1PO,Website,111642.110000,1.0
2,AR,3PO,DoorDash,73393.180000,1.0
3,AR,3PO,UberEats,2787.220000,1.0
4,KS,1PO,Mobile,289180.994286,14.0


In [386]:

# 3. Revenue share
total_rev_per_channel = mean_rev.groupby('channel')['revenue'].transform('sum')
total_rev_per_channel

mean_rev['revenue_share_label'] = mean_rev['Mean Revenue Share(%)'].map(lambda x: f"{x:.1f}%")

# 4. Plot with custom colors
fig = px.bar(mean_rev, 
             x="ST", 
             y="Mean Revenue Share(%)", 
             color="OrderMode",
             text='revenue_share_label',
             title="Little Caesars Mean Revenue Share(%) by State and Channel",
             labels={"ST": "State"},
             facet_row="channel",
            #  color_discrete_map=OrderMode_c,  # <-- Use custom colors here
             height=700,
             width=900)

fig.update_layout(
    yaxis_title="Mean Revenue Share(%)",
    showlegend=True
)

# Adjust font size for various elements
fig.update_layout(
    yaxis_title="Revenue Share per Store (%)",
    showlegend=True,
    title_font=dict(size=16),  # Title font size
    font=dict(size=10),  # Axis labels and general text size
    xaxis_title_font=dict(size=12),  # X-axis label font size
    yaxis_title_font=dict(size=12),  # Y-axis label font size
    legend_title_font=dict(size=12),  # Legend title font size
    legend_font=dict(size=10),  # Legend text size
)
fig.update_yaxes(matches=None)
fig.update_yaxes(showticklabels=True)  # Ensure y-axis labels show on one axis
fig.update_yaxes(title_text="", row=1, col=1)  # Remove y-axis title for each facet


fig.show()



0     3.921738e+06
1     3.921738e+06
2     1.577928e+06
3     1.577928e+06
4     3.921738e+06
5     3.921738e+06
6     1.577928e+06
7     1.577928e+06
8     1.577928e+06
9     3.921738e+06
10    3.921738e+06
11    1.577928e+06
12    1.577928e+06
13    1.577928e+06
14    3.921738e+06
15    3.921738e+06
16    1.577928e+06
17    1.577928e+06
18    1.577928e+06
19    3.921738e+06
20    3.921738e+06
21    1.577928e+06
22    1.577928e+06
23    1.577928e+06
24    3.921738e+06
25    3.921738e+06
26    1.577928e+06
27    1.577928e+06
28    1.577928e+06
29    3.921738e+06
30    3.921738e+06
31    1.577928e+06
32    1.577928e+06
33    1.577928e+06
34    3.921738e+06
35    3.921738e+06
36    1.577928e+06
37    1.577928e+06
38    1.577928e+06
39    3.921738e+06
40    3.921738e+06
41    1.577928e+06
42    1.577928e+06
43    1.577928e+06
Name: revenue, dtype: float64

In [387]:
mean_rev['total']=total_rev_per_channel
mean_rev


,ST,channel,OrderMode,revenue,store_count,total
0,AR,1PO,Mobile,217804.880000,1.0,3.921738e+06
1,AR,1PO,Website,111642.110000,1.0,3.921738e+06
2,AR,3PO,DoorDash,73393.180000,1.0,1.577928e+06
3,AR,3PO,UberEats,2787.220000,1.0,1.577928e+06
4,KS,1PO,Mobile,289180.994286,14.0,3.921738e+06
5,KS,1PO,Website,173604.558571,14.0,3.921738e+06
6,KS,3PO,DoorDash,91239.170000,14.0,1.577928e+06
7,KS,3PO,Grubhub,4452.394444,14.0,1.577928e+06
8,KS,3PO,UberEats,14711.703571,14.0,1.577928e+06
9,LA,1PO,Mobile,173505.980000,2.0,3.921738e+06


In [388]:
mean_rev['Mean Revenue Share(%)']=(mean_rev['revenue']/mean_rev['total'])*100
mean_rev


,ST,channel,OrderMode,revenue,store_count,total,Mean Revenue Share(%)
0,AR,1PO,Mobile,217804.880000,1.0,3.921738e+06,5.553784
1,AR,1PO,Website,111642.110000,1.0,3.921738e+06,2.846751
2,AR,3PO,DoorDash,73393.180000,1.0,1.577928e+06,4.651236
3,AR,3PO,UberEats,2787.220000,1.0,1.577928e+06,0.176638
4,KS,1PO,Mobile,289180.994286,14.0,3.921738e+06,7.373797
5,KS,1PO,Website,173604.558571,14.0,3.921738e+06,4.426725
6,KS,3PO,DoorDash,91239.170000,14.0,1.577928e+06,5.782212
7,KS,3PO,Grubhub,4452.394444,14.0,1.577928e+06,0.282167
8,KS,3PO,UberEats,14711.703571,14.0,1.577928e+06,0.932343
9,LA,1PO,Mobile,173505.980000,2.0,3.921738e+06,4.424211


In [389]:

mean_rev['revenue_share_label'] = mean_rev['Mean Revenue Share(%)'].map(lambda x: f"{x:.1f}%")

# 4. Plot with custom colors
fig = px.bar(mean_rev, 
             x="ST", 
             y="Mean Revenue Share(%)", 
             color="OrderMode",
             text='revenue_share_label',
             title="Little Caesars Mean Revenue Share(%) by State and Channel",
             labels={"ST": "State"},
             facet_row="channel",
            #  color_discrete_map=OrderMode_c,  # <-- Use custom colors here
             height=700,
             width=900)

fig.update_layout(
    yaxis_title="Mean Revenue Share(%)",
    showlegend=True
)

# Adjust font size for various elements
fig.update_layout(
    yaxis_title="Revenue Share per Store (%)",
    showlegend=True,
    title_font=dict(size=16),  # Title font size
    font=dict(size=10),  # Axis labels and general text size
    xaxis_title_font=dict(size=12),  # X-axis label font size
    yaxis_title_font=dict(size=12),  # Y-axis label font size
    legend_title_font=dict(size=12),  # Legend title font size
    legend_font=dict(size=10),  # Legend text size
)
fig.update_yaxes(matches=None)
fig.update_yaxes(showticklabels=True)  # Ensure y-axis labels show on one axis
fig.update_yaxes(title_text="", row=1, col=1)  # Remove y-axis title for each facet


fig.show()


In [322]:

aux[['Brand','ST','channel','OrderMode','mean_revenue_share','revenue_share_label','revenue_per_store']].groupby(['Brand','ST','channel','OrderMode'])


,Brand,ST,channel,OrderMode,StoreID,revenue,store_count_x,store_count_y,revenue_per_store,mean_revenue_share,revenue_share_label
0,LC,AR,1PO,Mobile,01494-00005,217804.88,1,1,217804.880000,5.553784,5.6%
1,LC,AR,1PO,Website,01494-00005,111642.11,1,1,111642.110000,2.846751,2.8%
2,LC,AR,3PO,DoorDash,01494-00005,73393.18,1,1,73393.180000,4.676589,4.7%
3,LC,AR,3PO,UberEats,01494-00005,2787.22,1,1,2787.220000,0.177601,0.2%
4,LC,KS,1PO,Mobile,03222-00009,141294.58,14,14,10092.470000,0.257347,0.3%
...,...,...,...,...,...,...,...,...,...,...,...
339,LC,VA,3PO,Grubhub,01978-00002,18023.93,3,3,6007.976667,0.382826,0.4%
340,LC,VA,3PO,Grubhub,01978-00005,23294.71,3,3,7764.903333,0.494777,0.5%
341,LC,VA,3PO,UberEats,01940-00009,87050.60,3,3,29016.866667,1.848945,1.8%
342,LC,VA,3PO,UberEats,01978-00002,90127.71,3,3,30042.570000,1.914303,1.9%


In [319]:


# Optional: Custom colors
OrderMode_c = {
    'Grubhub': '#EF553B',
    'Mobile': '#636EFA',
    'UberEats': '#FF6692',
    'DoorDash': '#FFA15A',
    'Website': 'rgb(68, 170, 153)',
    'Online': 'rgb(144, 103, 167)'
}
# Creating the plot
fig = px.bar(
    aux,
    x="ST",
    y="mean_revenue_share",
    color="OrderMode",
    text='revenue_share_label',
    title="State Revenue Share by Order Mode (Faceted by Channel and Brand)",
    labels={"ST": "State", "revenue_share": "Revenue Share (%)"},
   
    facet_col="Brand",
     facet_row="channel",
    color_discrete_map=OrderMode_c,
    height=800,
    width=1000
)

fig.update_layout(
    yaxis_title="Revenue Share (%)",
    showlegend=True
)

fig.update_yaxes(matches=None)

fig.show()


In [306]:
import pandas as pd
import plotly.express as px

# 1. Group revenue per state/channel/OrderMode/store
aux = x[x['Brand'] == 'LC'].groupby(['Brand', 'ST', 'channel', 'OrderMode'], as_index=False)['revenue'].sum()

# 2. Get store count per state
store_counts = x[x['Brand'] == 'LC'].groupby('ST')['StoreID'].nunique().reset_index()
store_counts.rename(columns={'StoreID': 'store_count'}, inplace=True)

# 3. Merge store count into revenue data
aux = aux.merge(store_counts, on='ST', how='left')

# 4. Normalize revenue by store count
aux['revenue_per_store'] = aux['revenue'] / aux['store_count']

# 5. Calculate total revenue_per_store within each Brand+State+Channel group
total_rev_per_group = aux.groupby(['Brand', 'ST', 'channel'])['revenue_per_store'].transform('sum')

# 6. Calculate revenue share (% of normalized revenue)
aux['revenue_share_percent'] = (aux['revenue_per_store'] / total_rev_per_group) * 100
aux['revenue_share_label'] = aux['revenue_share_percent'].map(lambda x: f"{x:.1f}%")

# 7. Custom color palette
OrderMode_c = {
    'Grubhub': '#EF553B',
    'Mobile': '#636EFA',
    'UberEats': '#FF6692',
    'DoorDash': '#FFA15A',
    'Website': 'rgb(68, 170, 153)',
    'Online': 'rgb(144, 103, 167)'
}

# 8. Plot revenue share (% normalized by store count)
fig = px.bar(
    aux,
    x="ST",
    y="revenue_share_percent",
    color="OrderMode",
    text="revenue_share_label",
    title="Normalized Revenue Share per Store by State, Channel, and Order Mode",
    labels={"ST": "State", "revenue_share_percent": "Revenue Share (%)"},
    facet_row="channel",
    facet_col="Brand",
    color_discrete_map=OrderMode_c,
    height=800,
    width=1000
)

fig.update_layout(
    yaxis_title="Normalized Revenue Share (%)",
    showlegend=True
)

fig.update_yaxes(matches=None)

fig.show()


In [185]:
lc_location=pd.read_excel('VIBE_LC locations.xlsx')
lc_location.columns


Index(['Zone', 'Franchise Store', 'Location', 'CaesarVision Store Name',
       'Address 1', 'City', 'ST', 'Postal\nCode', 'Latitude', 'Longitude',
       'Hours SUN_THU\nSUN-THU', 'Hours FRI_SAT\nFRI-SAT', 'Shared Mail',
       'DMA', 'County/Parish', 'Franchise Key Contact Email Name', 'Franchise',
       'Franchise Key Contact Email', 'Franchise Key Contact Phone Name',
       'Franchise Key Contact Phone ', 'AADT', 'AADT Source', 'ariel comments',
       'street concat', 'zone'],
      dtype='object')

In [186]:
lc_location['Franchise Store'].unique()


array([1389.0001, 1389.0002, 1389.0003, 1389.0004, 1389.0005, 1389.0008,
       1389.001 , 1389.0011, 1389.0012, 1389.0013, 1389.0014, 1494.0001,
       1494.0002, 1494.0004, 1494.0005, 1826.0001, 1826.0002, 1826.0003,
       1826.0005, 1893.0001, 1893.0002, 1940.0001, 1940.0007, 1940.0009,
       1940.001 , 1940.0011, 1940.0014, 1940.0021, 1944.0001, 1944.0003,
       1944.0004, 1944.0005, 1944.0006, 1978.0002, 1978.0005, 3222.0001,
       3222.0002, 3222.0003, 3222.0004, 3222.0006, 3222.0007, 3222.0008,
       3222.0009, 3222.001 , 3222.0012, 3222.0013, 3222.0014, 3222.0015,
       3222.0016, 3222.0017, 3222.0018, 3222.0019, 3222.0021, 3222.0022,
       3222.0023, 3222.0024, 3222.0025, 3222.0026, 3222.0027, 3222.0028,
       3222.0032, 3222.0033, 3222.0034, 3222.0035, 3222.0036, 3222.0038,
       3222.0039, 3222.0043, 3222.0044, 3222.0045, 3222.0046, 3222.0047,
       3646.0001, 3646.0002, 3646.0003, 3646.0004])

In [187]:
# Ensure the column is float (if it isn't already)
lc_location['Franchise Store'] = lc_location['Franchise Store'].astype(float)

# Split the number into integer and decimal parts
lc_location['store_id_formatted'] =lc_location['Franchise Store'] .apply(
    lambda x: f"{int(x):05d}-{int(round((x - int(x)) * 10000)):05d}"
)
lc_location['store_id_formatted'].unique()


array(['01389-00001', '01389-00002', '01389-00003', '01389-00004',
       '01389-00005', '01389-00008', '01389-00010', '01389-00011',
       '01389-00012', '01389-00013', '01389-00014', '01494-00001',
       '01494-00002', '01494-00004', '01494-00005', '01826-00001',
       '01826-00002', '01826-00003', '01826-00005', '01893-00001',
       '01893-00002', '01940-00001', '01940-00007', '01940-00009',
       '01940-00010', '01940-00011', '01940-00014', '01940-00021',
       '01944-00001', '01944-00003', '01944-00004', '01944-00005',
       '01944-00006', '01978-00002', '01978-00005', '03222-00001',
       '03222-00002', '03222-00003', '03222-00004', '03222-00006',
       '03222-00007', '03222-00008', '03222-00009', '03222-00010',
       '03222-00012', '03222-00013', '03222-00014', '03222-00015',
       '03222-00016', '03222-00017', '03222-00018', '03222-00019',
       '03222-00021', '03222-00022', '03222-00023', '03222-00024',
       '03222-00025', '03222-00026', '03222-00027', '03222-000

In [188]:
lc_location.columns


Index(['Zone', 'Franchise Store', 'Location', 'CaesarVision Store Name',
       'Address 1', 'City', 'ST', 'Postal\nCode', 'Latitude', 'Longitude',
       'Hours SUN_THU\nSUN-THU', 'Hours FRI_SAT\nFRI-SAT', 'Shared Mail',
       'DMA', 'County/Parish', 'Franchise Key Contact Email Name', 'Franchise',
       'Franchise Key Contact Email', 'Franchise Key Contact Phone Name',
       'Franchise Key Contact Phone ', 'AADT', 'AADT Source', 'ariel comments',
       'street concat', 'zone', 'store_id_formatted'],
      dtype='object')

In [189]:
df=lc_location[['ST','store_id_formatted','City','Zone','Latitude', 'Longitude']]
df['StoreID']=df['store_id_formatted']


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [192]:
lc_extended=lc.merge(df,on='StoreID',how='left')


In [124]:
import plotly.graph_objects as go
import pandas as pd

# List of all US state abbreviations
us_states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

# Aggregate your data
state_rev = lc_extended[lc_extended['channel']=='1PO'].groupby('ST')['revenue'].sum().reset_index()

# Fill in missing states with 0 revenue
all_states_df = pd.DataFrame({'ST': us_states})
state_rev = all_states_df.merge(state_rev, on='ST', how='left').fillna(0)
state_rev['revenue_text'] = state_rev['revenue'].apply(lambda x: f'{x:.2f}%')

# Store marker data
store_data = lc_extended[['StoreID', 'Latitude', 'Longitude', 'revenue']].dropna()
store_data = store_data.groupby(['StoreID', 'Latitude', 'Longitude'])['revenue'].sum().reset_index()

# Choropleth
choropleth = go.Choropleth(
    locations=state_rev['ST'],
    z=state_rev['revenue'],
    locationmode='USA-states',
    colorscale='Blues',
    colorbar_title='Revenue',
    text=state_rev['revenue_text'],
    hoverinfo="text",
    name='State Revenue'
   
)

# Store Markers
store_markers = go.Scattergeo(
    lon=store_data['Longitude'],
    lat=store_data['Latitude'],
    text=store_data.apply(lambda row: f"Store: {row['StoreID']}<br>Revenue: {row['revenue']}", axis=1),
    mode='markers',
    marker=dict(size=5, color='red', opacity=0.7),
    name='Stores'
)

# Combine layers
fig = go.Figure(data=[choropleth, store_markers])

fig.update_layout(
    title='LC 1PO Revenue by State with Store Locations',
    geo=dict(
        scope='usa',
        showland=True,
        landcolor='white',
        projection_type='albers usa'
    )
)

fig.show()


In [439]:
ws_extended[ws_extended['ST']=='SD']


,StoreID,OrderMode,WeekStartDate,WeeklyNetAmount,channel,Brand,revenue,ST,cleaned_ids,City
1680,778,DoorDash,2025-02-24,2285.98,3PO,WS,2285.98,SD,778,Sioux Falls
1681,778,DoorDash,2025-03-03,832.16,3PO,WS,832.16,SD,778,Sioux Falls
1682,778,DoorDash,2025-03-10,439.63,3PO,WS,439.63,SD,778,Sioux Falls
1683,778,DoorDash,2025-03-17,107.48,3PO,WS,107.48,SD,778,Sioux Falls
1684,778,Online,2025-02-24,6834.66,1PO,WS,6834.66,SD,778,Sioux Falls
1685,778,Online,2025-03-03,9041.01,1PO,WS,9041.01,SD,778,Sioux Falls
1686,778,Online,2025-03-10,9060.80,1PO,WS,9060.80,SD,778,Sioux Falls
1687,778,Online,2025-03-17,3027.22,1PO,WS,3027.22,SD,778,Sioux Falls
1688,778,UberEats,2025-02-24,1435.06,3PO,WS,1435.06,SD,778,Sioux Falls
1689,778,UberEats,2025-03-03,2600.69,3PO,WS,2600.69,SD,778,Sioux Falls


In [437]:
import plotly.graph_objects as go
import pandas as pd

# List of all US state abbreviations
us_states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

# Aggregate your data
state_rev = ws_extended[ws_extended['channel']=='3PO'].groupby('ST')['revenue'].sum().reset_index()

# Fill in missing states with 0 revenue
all_states_df = pd.DataFrame({'ST': us_states})
state_rev = all_states_df.merge(state_rev, on='ST', how='left').fillna(0)
state_rev['revenue_text'] = state_rev['revenue'].apply(lambda x: f'{x:.2f}%')

# Store marker data
store_data = lc_extended[['StoreID', 'Latitude', 'Longitude', 'revenue']].dropna()
store_data = store_data.groupby(['StoreID', 'Latitude', 'Longitude'])['revenue'].sum().reset_index()

# Choropleth
choropleth = go.Choropleth(
    locations=state_rev['ST'],
    z=state_rev['revenue'],
    locationmode='USA-states',
    colorscale='Blues',
    colorbar_title='Revenue',
    text=state_rev['revenue_text'],
    hoverinfo="text",
    name='State Revenue'
   
)

# Store Markers
store_markers = go.Scattergeo(
    lon=store_data['Longitude'],
    lat=store_data['Latitude'],
    text=store_data.apply(lambda row: f"Store: {row['StoreID']}<br>Revenue: {row['revenue']}", axis=1),
    mode='markers',
    marker=dict(size=5, color='red', opacity=0.7),
    name='Stores'
)

# Combine layers
fig = go.Figure(data=[choropleth, store_markers])

fig.update_layout(
    title='LC 3PO Revenue by State with Store Locations',
    geo=dict(
        scope='usa',
        showland=True,
        landcolor='white',
        projection_type='albers usa'
    )
)

fig.show()


In [127]:
import plotly.graph_objects as go
import pandas as pd

# Filter data for the 'Online' channel
channel_1 = '3PO'

# Filter data by 'Online' channel
channel_data_1 = lc_extended[lc_extended['channel'] == channel_1]

# Aggregate data per state for the 'Online' channel
state_rev_1 = channel_data_1.groupby('ST')['revenue'].sum().reset_index()

# Fill in missing states with 0 revenue
all_states_df = pd.DataFrame({'ST': us_states})
state_rev_1 = all_states_df.merge(state_rev_1, on='ST', how='left').fillna(0)

# Add formatted text for revenue
state_rev_1['revenue_text'] = state_rev_1['revenue'].apply(lambda x: f'{x:.2f}')

# Find the top 3 performing stores for the 'Online' channel based on revenue
top_stores = channel_data_1.groupby(['StoreID', 'Latitude', 'Longitude'])['revenue'].sum().reset_index()
top_stores_sorted = top_stores.sort_values(by='revenue', ascending=False).head(3)

# Create Choropleth for the 'Online' channel
choropleth_1 = go.Choropleth(
    locations=state_rev_1['ST'],
    z=state_rev_1['revenue'],
    locationmode='USA-states',
    colorscale='Blues',
    colorbar_title='Revenue (Online Channel)',
    text=state_rev_1['revenue_text'],
    hoverinfo="text",
    name=f'{channel_1} Revenue'
)

# Create Store Markers for the top 3 stores
store_markers = go.Scattergeo(
    lon=top_stores_sorted['Longitude'],
    lat=top_stores_sorted['Latitude'],
    text=top_stores_sorted.apply(lambda row: f"Store: {row['StoreID']}<br>Revenue: {row['revenue']}", axis=1),
    mode='markers+text',  # Adding text labels for store info
    marker=dict(size=10, color='red', opacity=0.7),
    name='Top 3 Stores'
)

# Combine Layers
fig = go.Figure(data=[choropleth_1, store_markers])

# Update layout with a larger size
fig.update_layout(
    title='Top 3 Performing Stores in Online Channel with Revenue by State',
    geo=dict(
        scope='usa',
        showland=True,
        landcolor='white',
        projection_type='albers usa'
    ),
    height=800,  # Increase height for larger view
    width=1200,  # Increase width for larger view
    margin={"r":0,"t":40,"l":0,"b":0}  # Adjust margins for better spacing
)

fig.show()


In [243]:
import plotly.express as px

print(px.colors.qualitative.Plotly[1])


#EF553B


In [332]:
import pandas as pd
import pandas as pd
import plotly.express as px

# OrderMode_c = {
#     'Grubhub': '#EF553B',   # Muted Blue
#     'Mobile': '#636EFA',     # Soft Orange
#     'UberEats': '#FF6692',   # Muted Green
#     'DoorDash': '#FFA15A',    # Coral Red
#     'Website': 'rgb(68, 170, 153)',   # Neutral Gray
#     'Online': 'rgb(144, 103, 167)'     # Plum Purple
# }
# 2. Group data
aux = lc_extended.groupby(['ST', 'channel', 'OrderMode'], as_index=False)['revenue'].sum()
aux


,ST,channel,OrderMode,revenue
0,AR,1PO,Mobile,217804.88
1,AR,1PO,Website,111642.11
2,AR,3PO,DoorDash,73393.18
3,AR,3PO,UberEats,2787.22
4,KS,1PO,Mobile,4048533.92
5,KS,1PO,Website,2430463.82
6,KS,3PO,DoorDash,1277348.38
7,KS,3PO,Grubhub,40071.55
8,KS,3PO,UberEats,205963.85
9,LA,1PO,Mobile,347011.96


In [247]:

# 3. Revenue share
total_rev_per_channel = aux.groupby('channel')['revenue'].transform('sum')
aux['revenue_share'] = aux['revenue'] / total_rev_per_channel * 100
aux['revenue_share_label'] = aux['revenue_share'].map(lambda x: f"{x:.1f}%")

# 4. Plot with custom colors
fig = px.bar(aux, 
             x="ST", 
             y="revenue_share", 
             color="OrderMode",
             text='revenue_share_label',
             title="State Revenue Share by Order Mode (Faceted by Channel)",
             labels={"ST": "State", "revenue_share": "Revenue Share (%)"},
             facet_row="channel",
            #  color_discrete_map=OrderMode_c,  # <-- Use custom colors here
             height=800,
             width=1000)

fig.update_layout(
    yaxis_title="Revenue Share (%)",
    showlegend=True
)

fig.update_yaxes(matches=None)

fig.show()



In [436]:
store_counts


,ST,store_count
0,AR,1
1,KS,14
2,LA,2
3,MD,6
4,NC,7
5,NM,6
6,OK,17
7,TX,20
8,VA,3


In [435]:

import pandas as pd
import plotly.express as px

# 1. Filter for Little Caesars and calculate revenue per state/channel/ordermode
aux = x[x['Brand'] == 'WS'].groupby(['Brand', 'ST', 'channel', 'OrderMode','StoreID'], as_index=False)['revenue'].sum()

# 2. Get store count per state (assuming 'store_id' column exists in your dataset)
store_counts = x[x['Brand'] == 'LC'].groupby('ST')['StoreID'].nunique().reset_index()
store_counts.rename(columns={'StoreID': 'store_count'}, inplace=True)
store_counts
# 3. Merge store count into the main data
aux = aux.merge(store_counts, on='ST', how='left')
aux.head()
# 4. Calculat
mean_rev=aux.groupby(['ST','channel','OrderMode']).mean().reset_index()
mean_rev.head()
# 3. Revenue share
total_rev_per_channel = mean_rev.groupby('channel')['revenue'].transform('sum')
total_rev_per_channel
mean_rev['total']=total_rev_per_channel
mean_rev
mean_rev['Mean Revenue Share(%)']=(mean_rev['revenue']/mean_rev['total'])*100
mean_rev


,ST,channel,OrderMode,StoreID,revenue,store_count,total,Mean Revenue Share(%)
0,AR,1PO,Online,1566.000000,704159.490000,1.0,6.890761e+06,10.218892
1,AR,3PO,DoorDash,1566.000000,452365.183333,1.0,5.544352e+06,8.159027
2,AR,3PO,UberEats,1566.000000,156122.550000,1.0,5.544352e+06,2.815884
3,KS,1PO,Online,1951.833333,711018.223333,14.0,6.890761e+06,10.318428
4,KS,3PO,DoorDash,1951.833333,439301.281667,14.0,5.544352e+06,7.923402
5,KS,3PO,UberEats,1951.833333,101436.756667,14.0,5.544352e+06,1.829551
6,MO,1PO,Online,2309.500000,597269.155000,NaN,6.890761e+06,8.667680
7,MO,3PO,DoorDash,2309.500000,452225.715000,NaN,5.544352e+06,8.156511
8,MO,3PO,UberEats,2309.500000,49462.615000,NaN,5.544352e+06,0.892126
9,NC,1PO,Online,2207.000000,693848.810000,7.0,6.890761e+06,10.069262


In [382]:


mean_rev['revenue_share_label'] = mean_rev['Mean Revenue Share(%)'].map(lambda x: f"{x:.1f}%")

# 4. Plot with custom colors
fig = px.bar(mean_rev, 
             x="ST", 
             y="Mean Revenue Share(%)", 
             color="OrderMode",
             text='revenue_share_label',
             title="Wingstop Mean Revenue Share(%) by State and Channel",
             labels={"ST": "State"},
             facet_row="channel",
            #  color_discrete_map=OrderMode_c,  # <-- Use custom colors here
             height=700,
             width=900)

fig.update_layout(
    yaxis_title="Mean Revenue Share(%)",
    showlegend=True
)

# Adjust font size for various elements
fig.update_layout(
    yaxis_title="Revenue Share per Store (%)",
    showlegend=True,
    title_font=dict(size=16),  # Title font size
    font=dict(size=10),  # Axis labels and general text size
    xaxis_title_font=dict(size=12),  # X-axis label font size
    yaxis_title_font=dict(size=12),  # Y-axis label font size
    legend_title_font=dict(size=12),  # Legend title font size
    legend_font=dict(size=10),  # Legend text size
)
fig.update_yaxes(matches=None)
fig.update_yaxes(showticklabels=True)  # Ensure y-axis labels show on one axis
fig.update_yaxes(title_text="", row=1, col=1)  # Remove y-axis title for each facet


fig.show()



In [430]:
import pandas as pd
import plotly.express as px

# 1. Filter for Little Caesars and calculate revenue per state/channel/ordermode
aux = x[x['Brand'] == 'WS'].groupby(['Brand', 'ST', 'channel', 'OrderMode','StoreID'], as_index=False)['revenue'].sum()

# 2. Get store count per state (assuming 'store_id' column exists in your dataset)
store_counts = x[x['Brand'] == 'WS'].groupby('ST')['StoreID'].nunique().reset_index()
store_counts.rename(columns={'StoreID': 'store_count'}, inplace=True)
store_counts
# 3. Merge store count into the main data
aux = aux.merge(store_counts, on='ST', how='left')
aux=aux[['Brand','ST','channel','OrderMode','revenue']]
aux.shape
aux=aux.groupby(['Brand','ST',	'channel',	'OrderMode']).sum().reset_index()
total_rev_per_state= aux.groupby('ST')['revenue'].transform('sum')
total_rev_per_state
aux['total_rev_per_state']=total_rev_per_state
aux[aux['ST']=='VA']


aux['revenue Share(%)']=(aux['revenue']/aux['total_rev_per_state'])*100
aux


aux['revenue_share_label'] = aux['revenue Share(%)'].map(lambda x: f"{x:.1f}%")

aux


# 4. Plot with custom colors
fig = px.bar(aux, 
             x="ST", 
             y="revenue Share(%)", 
             color="OrderMode",
             text='revenue_share_label',
             title="Wingstop Revenue Share(%) by State",
             labels={"ST": "State"},
             facet_row="channel",
            #  color_discrete_map=OrderMode_c,  # <-- Use custom colors here
             height=700,
             width=900)

fig.update_layout(
    yaxis_title="Revenue Share(%)",
    showlegend=True
)

# Adjust font size for various elements
fig.update_layout(
    yaxis_title="Share per Store (%)",
    showlegend=True,
    title_font=dict(size=16),  # Title font size
    font=dict(size=10),  # Axis labels and general text size
    xaxis_title_font=dict(size=12),  # X-axis label font size
    yaxis_title_font=dict(size=12),  # Y-axis label font size
    legend_title_font=dict(size=12),  # Legend title font size
    legend_font=dict(size=10),  # Legend text size
)
fig.update_yaxes(matches=None)
fig.update_yaxes(showticklabels=True)  # Ensure y-axis labels show on one axis
fig.update_yaxes(title_text="", row=1, col=1)  # Remove y-axis title for each facet


fig.show()





In [431]:
import pandas as pd
import plotly.express as px

# 1. Filter for Little Caesars and calculate revenue per state/channel/ordermode
aux = x[x['Brand'] == 'LC'].groupby(['Brand', 'ST', 'channel', 'OrderMode','StoreID'], as_index=False)['revenue'].sum()

# 2. Get store count per state (assuming 'store_id' column exists in your dataset)
store_counts = x[x['Brand'] == 'LC'].groupby('ST')['StoreID'].nunique().reset_index()
store_counts.rename(columns={'StoreID': 'store_count'}, inplace=True)
store_counts
# 3. Merge store count into the main data
aux = aux.merge(store_counts, on='ST', how='left')
aux=aux[['Brand','ST','channel','OrderMode','revenue']]
aux.shape
aux=aux.groupby(['Brand','ST',	'channel',	'OrderMode']).sum().reset_index()
total_rev_per_state= aux.groupby('ST')['revenue'].transform('sum')
total_rev_per_state
aux['total_rev_per_state']=total_rev_per_state
aux[aux['ST']=='VA']


aux['revenue Share(%)']=(aux['revenue']/aux['total_rev_per_state'])*100
aux


aux['revenue_share_label'] = aux['revenue Share(%)'].map(lambda x: f"{x:.1f}%")

aux


# 4. Plot with custom colors
fig = px.bar(aux, 
             x="ST", 
             y="revenue Share(%)", 
             color="OrderMode",
             text='revenue_share_label',
             title="Little Caesars Revenue Share(%) by State",
             labels={"ST": "State"},
             facet_row="channel",
            #  color_discrete_map=OrderMode_c,  # <-- Use custom colors here
             height=700,
             width=900)

fig.update_layout(
    yaxis_title="Revenue Share(%)",
    showlegend=True
)

# Adjust font size for various elements
fig.update_layout(
    yaxis_title="Share per Store (%)",
    showlegend=True,
    title_font=dict(size=16),  # Title font size
    font=dict(size=10),  # Axis labels and general text size
    xaxis_title_font=dict(size=12),  # X-axis label font size
    yaxis_title_font=dict(size=12),  # Y-axis label font size
    legend_title_font=dict(size=12),  # Legend title font size
    legend_font=dict(size=10),  # Legend text size
)
fig.update_yaxes(matches=None)
fig.update_yaxes(showticklabels=True)  # Ensure y-axis labels show on one axis
fig.update_yaxes(title_text="", row=1, col=1)  # Remove y-axis title for each facet


fig.show()



